<a href="https://colab.research.google.com/github/jungfrau70/project1-masksdetection/blob/main/masks_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Masks Detection Web Service

참조

https://masks-detection-13237.web.app

https://github.com/ohyicong/masksdetection.git

# Download Code from GITHUB

https://github.com/jungfrau70/project1-masksdetection.git

In [2]:
!git clone https://github.com/jungfrau70/project1-masksdetection.git

Cloning into 'masksdetection'...
remote: Enumerating objects: 3228, done.
remote: Counting objects: 100% (725/725), done.
remote: Compressing objects: 100% (687/687), done.
remote: Total 3228 (delta 36), reused 715 (delta 31), pack-reused 2503
Receiving objects: 100% (3228/3228), 103.31 MiB | 41.93 MiB/s, done.
Resolving deltas: 100% (311/311), done.


## STEP 1. import

In [1]:
# import the necessary packages

import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import tensorflow as tf

from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## STEP 2. 전처리 (pre-processing)

In [3]:
print(os.getcwd())
!ls -al project1-masksdetection

/content
ls: cannot access 'project1-masksdetection': No such file or directory


In [ ]:
import os
os.chdir('project1-masksdetection')
!ls -al .

total 88
drwxr-xr-x 12 root root 4096 May 29 19:08 .
drwxr-xr-x  1 root root 4096 May 29 19:08 ..
-rw-r--r--  1 root root 5788 May 29 19:08 create_augmented_dataset.py
drwxr-xr-x  4 root root 4096 May 29 19:08 dataset
-rw-r--r--  1 root root 4848 May 29 19:08 detect_mask_video.py
-rw-r--r--  1 root root 1059 May 29 19:08 detect_mask_video.spec
drwxr-xr-x  2 root root 4096 May 29 19:08 face_detector
drwxr-xr-x  8 root root 4096 May 29 19:08 .git
-rw-r--r--  1 root root   28 May 29 19:08 .gitignore
drwxr-xr-x  2 root root 4096 May 29 19:08 images
drwxr-xr-x  2 root root 4096 May 29 19:08 .ipynb_checkpoints
drwxr-xr-x  3 root root 4096 May 29 19:09 model
drwxr-xr-x  2 root root 4096 May 29 19:08 plot
drwxr-xr-x  2 root root 4096 May 29 19:08 __pycache__
-rw-r--r--  1 root root 7445 May 29 19:08 requirements.txt
drwxr-xr-x  2 root root 4096 May 29 19:08 sounds
-rw-r--r--  1 root root 5134 May 29 19:08 train_mask_detector.py
drwxr-xr-x  4 root root 4096 May 29 19:08 web_demo


In [ ]:
!rm -rf model; mkdir model; ls -al model

In [ ]:
# construct the argument parser and parse the arguments
dataset_path=os.getcwd()+"//dataset"
model_path=os.getcwd()+"//model//mask_model"
plot_path=os.getcwd()+"//plot"

In [ ]:
training_datagen = ImageDataGenerator(
    rescale=1. / 255,
    # 위의 옵션 값들을 보고 적절히 대입하여 줍니다.
    rotation_range=20,
    zoom_range=0.15,  
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.15, 
    horizontal_flip=True, 
    fill_mode='nearest', 
    validation_split=0.2
    )

In [ ]:
TRAINING_DIR = "dataset/"

In [ ]:
training_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(224, 224),
                                                            color_mode='rgb', classes=None,
                                                            class_mode='categorical',
                                                            batch_size=128,                                                            
                                                            subset='training', 
)

Found 1131 images belonging to 2 classes.


In [ ]:
validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(224, 224),
                                                            color_mode='rgb', classes=None,
                                                            class_mode='categorical',
                                                            batch_size=128,                                                            
                                                            subset='validation', 
                                                            )

Found 282 images belonging to 2 classes.


## STEP 3. 모델 정의 (Sequential)


In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


## STEP 4. optimizer, loss

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

[INFO] compiling model...


## STEP 5. ModelCheckpoint

In [ ]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

## STEP 6. 학습 (fit)

In [ ]:
H = model.fit(training_generator,
          epochs=BS, 
          verbose=1, 
          callbacks=[checkpoint],
          validation_data=(validation_generator)
          )

Epoch 1/32
9/9 [==============================] - 50s 2s/step - loss: 0.7672 - acc: 0.5729 - val_loss: 0.3488 - val_acc: 0.9291

Epoch 00001: val_loss improved from inf to 0.34878, saving model to my_checkpoint.ckpt
Epoch 2/32
9/9 [==============================] - 16s 2s/step - loss: 0.3715 - acc: 0.8382 - val_loss: 0.1796 - val_acc: 0.9681

Epoch 00002: val_loss improved from 0.34878 to 0.17958, saving model to my_checkpoint.ckpt
Epoch 3/32
9/9 [==============================] - 16s 2s/step - loss: 0.1917 - acc: 0.9399 - val_loss: 0.1079 - val_acc: 0.9752

Epoch 00003: val_loss improved from 0.17958 to 0.10795, saving model to my_checkpoint.ckpt
Epoch 4/32
9/9 [==============================] - 16s 2s/step - loss: 0.1178 - acc: 0.9655 - val_loss: 0.0892 - val_acc: 0.9787

Epoch 00004: val_loss improved from 0.10795 to 0.08923, saving model to my_checkpoint.ckpt
Epoch 5/32
9/9 [==============================] - 16s 2s/step - loss: 0.0960 - acc: 0.9779 - val_loss: 0.0658 - val_acc: 0.9

## STEP 7. 학습 완료 후 Load Weights (ModelCheckpoint)

In [ ]:
model.load_weights(checkpoint_path)

NameError: ignored

## STEP 8. 모델 저장

In [ ]:
# serialize the model to disk
print("[INFO] saving mask detector model... path: %s"%(model_path+".h5"))
model.save(model_path+".h5")

## STEP 9. 예측 (Predict)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

## STEP 10. 리포트

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)